In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# GPT2-XL

In [22]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "openai-community/gpt2-xl",
    quantization_config=quantization_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")

In [23]:
prompt = """
Ты — эксперт по математическому анализу и спектральным методам. Твоя задача — найти оператор Лапласа - Бельтрами по заданному уравнению поверхности.
оператор Лапласа - Бельтрами для уравнения поверхности {} будет выражаться как 
Исходные данные для задачи:
- Уравнение поверхности : r(θ, ϕ) = (R sin ϕ cos θ, R sin ϕ sin θ, R cos ϕ)
- Область значений функции :  θ ∈ [0, 2π), ϕ ∈ [0, π]
"""
prompt = "The Laplace–Beltrami operator for the surface equation r(θ, ϕ) = (R sin ϕ cos θ, R sin ϕ sin θ, R cos ϕ) is expressed as"

In [24]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [25]:
print(answer)

The Laplace–Beltrami operator for the surface equation r(θ, ϕ) = (R sin ϕ cos θ, R sin ϕ sin θ, R cos ϕ) is expressed as:

where R is the radius of the Earth, ϕ is the Earth's angular velocity, and θ is the angle between the Earth's surface and the line of sight to the Earth. The Laplace–Beltrami operator is a special case of the Laplace operator for the surface equation r(θ, ϕ) = (R cos ϕ sin θ, R cos ϕ sin θ, R sin ϕ cos θ, R cos ϕ) , which is the same as the Laplace operator for the surface equation r(θ, ϕ) = (R sin ϕ cos θ, R sin ϕ sin θ, R cos ϕ) .

The Laplace operator for the surface equation r(θ, ϕ) = (R cos ϕ sin θ, R sin ϕ cos θ, R cos ϕ) is expressed as:

where R is the radius


# QWEN2-1.5b

In [26]:
import torch
from transformers import pipeline

In [29]:
pipe = pipeline(
    task="text-generation",
    model="Qwen/Qwen2-1.5B-Instruct",
    dtype=torch.bfloat16,
    device_map=0
)

messages = [
    {"role": "system", "content": "You are a mathematition."},
    {"role": "user", "content": "The Laplace–Beltrami operator for the surface equation r(θ, ϕ) = (R sin ϕ cos θ, R sin ϕ sin θ, R cos ϕ) is expressed as"},
]
outputs = pipe(messages, max_new_tokens=444, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
answer = outputs[0]["generated_text"][-1]['content']

Device set to use cuda:0


In [30]:
print(answer)

The Laplace-Beltrami operator on a surface of revolution can be expressed in terms of cylindrical coordinates. Let's denote the surface parameterization by:

r(θ, ϕ) = (R \* sin(\phi) \* cos(θ), R \* sin(\phi) \* sin(θ), R \* cos(\phi))

Then, we can write this parameterization in spherical coordinates as well:

r(θ, ϕ) = (R \* \sin(\theta) \* \cos(\phi), R \* \sin(\theta) \* \sin(\phi), R \* \cos(\theta))

Now, let's express the Laplacian in cylindrical and spherical coordinates:

1. **Laplacian in Cylindrical Coordinates**: For a surface parametrized by \(r\cdot r'\) + \(r''\) = 0,

∇^2 f = -∂f/∂r + ∂²f/∂r² + ∂²f/∂θ²

For our surface, this becomes:
∇^2 f = -∂f/∂θ + ∂²f/∂φ².

2. **Laplacian in Spherical Coordinates**: The Laplacian in spherical coordinates with respect to φ (angle between r and φ) is given by:

∇^2 f = -∂f/∂θ² + ∂²f/∂ϕ²

In our case, since φ = ϕ (same angle between r and φ at all points on the surface), we get:

∇^2 f = -∂f/∂θ² + ∂²f/∂θ² = -∂f/∂θ².

So, the Laplace-Be